In [11]:
# Packages installation.
!pip install deepface firebase-admin Pillow tensorflow==2.15.0


In [12]:
# Importing section
import os
from deepface import DeepFace
import firebase_admin
from firebase_admin import credentials, storage


In [13]:
# Load Firebase credentials and initialize the app with storage configuration
def initialize_firebase():
    if not firebase_admin._apps:
        cred = credentials.Certificate('auth.json')
        firebase_admin.initialize_app(cred, {'storageBucket': 'coldrecog.appspot.com'})
    return storage.bucket()


In [14]:
# Function to compare images and return the minimum match rate and corresponding file path
def compare_images(test_img_path, folder_path):
    # Access the default Cloud Storage bucket
    bucket = initialize_firebase()
    
    # List all the blobs (files) in the specified folder path
    blobs = bucket.list_blobs(prefix=folder_path)
    
    # Temporary file path for storing the downloaded reference image
    reference_img_path = "temp.jpg"
    
    # Dictionary to store the match rates for each image
    predict = {}

    # Loop through each blob in the specified folder
    for blob in blobs:
        try:
            # Get the file path of the current blob
            file_path = blob.name
            
            # Skip if the file path corresponds to the folder itself
            if file_path == folder_path + '/':
                continue
            
            # Download the image data from the current blob
            blob = bucket.blob(file_path)
            image_data = blob.download_as_bytes()

            # Save the downloaded image locally
            local_file_path = f'./{reference_img_path}'
            with open(local_file_path, 'wb') as local_file:
                local_file.write(image_data)

            # Use DeepFace library to verify similarity between reference and test images
            result = DeepFace.verify(reference_img_path, test_img_path)
            match_rate = round(result["distance"] * 100, 2)

            # Store the match rate in the dictionary with the file path as the key
            predict[file_path] = match_rate
            
            # Remove the temporary reference image file
            os.remove(reference_img_path)

        except Exception as e:
            print(f'Error: {e}')
            print("Loading.....")

    # Find the minimum match rate and corresponding file path(s)
    matched = min(predict.values())
    res = [key for key in predict if predict[key] == matched]
    return res[0], matched


In [18]:
# Define the folder path in the bucket
folder_path = 'aadhar'  # Replace with the correct folder path

# Get the test image path from the user
test_img_path = input("Enter the path to the test image: ")

# Call the function to compare images and get the result
matched_file, match_distance = compare_images(test_img_path, folder_path)
print("The Match is", matched_file, "with distance of", match_distance)


KeyboardInterrupt: Interrupted by user